In [1]:
import sys
import numpy as np

np.random.seed(10)

sys.path.append("../")
import pandas as pd
import os

from sklearn.model_selection import train_test_split
import plotly.express as px

from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

import src.preprocessing as pp
pd.get_option("display.max_rows")

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

import matplotlib.pyplot as plt
import sklearn.metrics as metrics

import lightgbm as lgb
import shap
from sklearn.model_selection import GridSearchCV

import src.visualisations as vis
import src.calculations as cl

c:\Users\hannah.alexander\Anaconda3\envs\speed_date_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
identifiers = ["date_id_a", "match_a", "wave_a", 
"date_id_b",  "wave_b"]

# same as 1_1
#preference = ["pf_o_att_a", "pf_o_sin_a", "pf_o_int_a", "pf_o_fun_a", "pf_o_amb_a", "pf_o_sha_a", 
#"pf_o_att_b", "pf_o_sin_b", "pf_o_int_b", "pf_o_fun_b", "pf_o_amb_b", "pf_o_sha_b"]

personal_info = ["field_cd_a", "age_a", "race_a", "income_a", "samerace_a", "goal_a",
"field_cd_b", "age_b", "race_b", "income_b", "goal_b", "distance_bin", "zipcode_area_b", "zipcode_area_a"]

behaviour = ["go_out_a", "sports_a", "tvsports_a", "exercise_a", "dining_a", "museums_a", "art_a", "hiking_a", "gaming_a",
           "clubbing_a", "reading_a", "tv_a", "theater_a", "movies_a", "concerts_a", "music_a", "shopping_a",
           "yoga_a", "exphappy_a",
           "go_out_b", "sports_b", "tvsports_b", "exercise_b", "dining_b", "museums_b", "art_b", "hiking_b", "gaming_b",
           "clubbing_b", "reading_b", "tv_b", "theater_b", "movies_b", "concerts_b", "music_b", "shopping_b",
           "yoga_b", "exphappy_b"]

difference_hobbies = ["go_out_df", "sports_df", "tvsports_df", "exercise_df", "dining_df", "museums_df", "art_df", "hiking_df", "gaming_df",
           "clubbing_df", "reading_df", "tv_df", "theater_df", "movies_df", "concerts_df", "music_df", "shopping_df", "yoga_df", "exphappy_df"]

hobbies_big_diff = ["go_out_df_big", "sports_df_big", "tvsports_df_big", "exercise_df_big", "dining_df_big", "museums_df_big", "art_df_big", "hiking_df_big", "gaming_df_big",
           "clubbing_df_big", "reading_df_big", "tv_df_big", "theater_df_big", "movies_df_big", "concerts_df_big", "music_df_big", "shopping_df_big", "yoga_df_big", "exphappy_df_big"]

looking_for = ["attr1_1_a", "sinc1_1_a", "intel1_1_a", "fun1_1_a", "amb1_1_a", "imprace_a", "imprelig_a",
"attr1_1_b", "sinc1_1_b", "intel1_1_b", "fun1_1_b", "amb1_1_b", "imprace_b", "imprelig_b"]

self_rating = [ "attr3_1_a", "sinc3_1_a", "intel3_1_a", "fun3_1_a", "amb3_1_a", "attr3_1_b", "sinc3_1_b", "intel3_1_b", "fun3_1_b", "amb3_1_b"]

# difference between what person a i slooking for b what person a rates themselves 1_1_a and 3_1_b
difference = ["attr_b_a", "sinc_b_a", "intel_b_a", "fun_b_a", "amb_b_a", "attr_a_b", "sinc_a_b", "intel_a_b", "fun_a_b", "amb_a_b"]

to_drop = ["date_id_a", "date_id_b", "wave_a", "wave_b"] #TODO clean text in from column , 

to_keep = personal_info + identifiers + difference + difference_hobbies + ["both_creative", "both_sporty"]#+ hobbies_big_diff

categorical = ["income_a", "income_b", "distance_bin", "zipcode_area_a", "zipcode_area_b", "race_a", "race_b"]

In [3]:
# read in data
df = pd.read_csv("../data/interim/single_row_date.csv")
df = df[to_keep]
df.drop(to_drop, axis = 1, inplace = True)

x = df.match_a.value_counts()
x[0]/(x[1]+x[0])

0.834248455730954

In [4]:
df = pd.get_dummies(df, columns = categorical)

In [5]:
df.columns [df.isnull().any()]

Index([], dtype='object')

In [6]:
import umap

def umap_transformation(df): 
    '''
    Genertaes a umap cluster plot 

    Arguments:
    df- dataset containing leads to be used 

    Returns:
    The clustered leads

    '''
    standard_embedding = umap.UMAP(random_state=42, n_neighbors=40, min_dist=0.1, metric = "chebyshev").fit(df) 
    return standard_embedding.embedding_

In [7]:
umap_data = umap_transformation(df)

In [8]:
px.scatter(x = list(zip(*umap_data))[0], y = list(zip(*umap_data))[1], height = 800, width = 1200)

In [17]:
db, df_umap = vis.dbscan_clustering(umap_data, df, eps = 0.7, min_samples = 50)

df_count = df_umap.groupby(["match_a", "labels"]).size().reset_index()
df_count_p = df_count.pivot(index = "labels", columns = ["match_a"])[0]
df_count_p.fillna(0, inplace = True)
# Display the breakdowns of conversion rates for each cluster 
df_count_p["Total"] = df_count_p[0] + df_count_p[1]
df_count_p["Percentage_matched"] = round((df_count_p[1]) / df_count_p["Total"] * 100, 2)
df_count_p

[-1  0  1  2  3  4  5  6  7  8  9]


match_a,0,1,Total,Percentage_matched
labels,,,,
-1,221,55,276,19.93
0,286,64,350,18.29
1,608,121,729,16.60
2,142,30,172,17.44
3,224,46,270,17.04
4,316,62,378,16.40
5,346,59,405,14.57
6,94,10,104,9.62
7,44,7,51,13.73


### Generating a full profile summary

In [18]:
import copy
clustered_df = copy.deepcopy(df)
clustered_df['labels'] = list(df_umap['labels'])

In [20]:
# Generate the full profile list of feature results for each cluster
df_profile = cl.full_summary(clustered_df)
df_profile

,metric,-1,0,1,2,3,4,5,6,7,8,9,Overall Dataset
column,,,,,,,,,,,,,
field_cd_a,mean,10.231884,3.568571,7.980727,13.691860,8.818519,9.986706,9.310216,7.894231,4.509804,7.854545,2.188406,8.317969
age_a,mean,24.654775,24.051429,25.165944,25.783690,26.149478,26.011938,26.055586,35.451923,32.882353,27.318182,32.666667,26.105652
samerace_a,mean,0.384058,0.411429,0.326475,0.406977,0.307407,0.425926,0.350617,0.509615,0.490196,0.409091,0.507246,0.378174
goal_a,mean,2.065217,2.122857,1.853861,2.261628,1.992593,1.681832,1.503344,2.923077,3.019608,5.090909,3.014493,2.080320
field_cd_b,mean,6.840580,8.248571,3.834019,7.633721,14.159259,9.563492,7.893827,8.240385,3.686275,8.545455,9.507246,7.604324
...,...,...,...,...,...,...,...,...,...,...,...,...,...
race_b_1.0,mean,0.010870,0.031429,0.063100,0.058140,0.062963,0.015873,0.037037,0.009615,0.117647,0.036364,0.057971,0.042210
race_b_2.0,mean,0.550725,0.691429,0.493827,0.686047,0.466667,0.724868,0.622222,0.644231,0.509804,0.627273,0.652174,0.594029
race_b_3.0,mean,0.047101,0.062857,0.063100,0.116279,0.074074,0.000000,0.083951,0.019231,0.156863,0.045455,0.043478,0.059369


In [21]:
# Outlining the continuous and catagorical features included in the full summary
continuous = ["attr_b_a", "sinc_b_a", "intel_b_a", "fun_b_a", "amb_b_a", "attr_a_b", "sinc_a_b", "intel_a_b", "fun_a_b", "amb_a_b", "go_out_df", "sports_df", "tvsports_df", "exercise_df", "dining_df", "museums_df", "art_df", "hiking_df", "gaming_df",
           "clubbing_df", "reading_df", "tv_df", "theater_df", "movies_df", "concerts_df", "music_df", "shopping_df", "yoga_df", "exphappy_df", "age_a", "age_b"]

catagorical = ["race_a", "race_b", "distance_bin", "zipcode_area_b", "zipcode_area_a", "samerace_a", "goal_a", "goal_b", "field_cd_a", "field_cd_b", "income_b", "income_a", "both_creative", "both_sporty"]

In [22]:
# Obtaining the most common profile for each cluster based on the results of the full profile 
results_table = cl.best_profiles(df_count_p, continuous, catagorical, df_profile)
results_table

,attr_b_a,sinc_b_a,intel_b_a,fun_b_a,amb_b_a,attr_a_b,sinc_a_b,intel_a_b,fun_a_b,amb_a_b,...,zipcode_area_a,samerace_a,goal_a,goal_b,field_cd_a,field_cd_b,income_b,income_a,both_creative,both_sporty
-1,0.388396,0.545150,0.408313,0.374892,0.533808,0.463550,0.439427,0.344313,0.332095,0.486353,...,_1,,,,,,_Not provided,_Not provided,,
0,0.373862,0.550335,0.412758,0.417544,0.539830,0.460547,0.488930,0.384979,0.365353,0.494877,...,_1,,,,,,_Not provided,_Not provided,,
1,0.418839,0.511267,0.364445,0.400726,0.543250,0.453077,0.503199,0.378381,0.395427,0.504771,...,_1,,,,,,_Not provided,_Not provided,,
2,0.397660,0.593512,0.336359,0.359591,0.518277,0.484099,0.501708,0.411898,0.383680,0.509432,...,_1,,,,,,_Not provided,_Not provided,,
3,0.448109,0.549738,0.379703,0.434919,0.545094,0.485288,0.450210,0.368903,0.339789,0.436580,...,_1,,,,,,_Not provided,_Not provided,,
4,0.435706,0.530930,0.408613,0.406558,0.546800,0.435551,0.444082,0.385936,0.293584,0.403624,...,_1,,,,,,_Not provided,_Not provided,,
5,0.445914,0.549487,0.391993,0.412316,0.563097,0.432968,0.478504,0.419983,0.407400,0.473148,...,_1,,,,,,_Not provided,_Not provided,,
6,0.449927,0.463654,0.352603,0.418479,0.321828,0.429613,0.466904,0.430711,0.350333,0.468740,...,_1,,,,,,_Not provided,_Not provided,,
7,0.426737,0.602520,0.354213,0.352314,0.407235,0.456841,0.402464,0.337439,0.421792,0.494623,...,_1,,,,,,_Not provided,_Not provided,,
8,0.530962,0.500036,0.409858,0.416676,0.544751,0.467055,0.471055,0.407253,0.333902,0.341765,...,_1,,,,,,_Not provided,_19998.0 and 29998.0,,
